<a href="https://colab.research.google.com/github/prakharsrivastava/powerbi/blob/main/Data_Science_on_Google_Cloud_Platform_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import datetime, pytz
def utc_convert(date, hhmm, tzone):
  loc_tz = pytz.timezone("America/Anchorage")
  loc_dt = loc_tz.localize(datetime.datetime.strptime("2015-04-28", '%Y-%m-%d'), is_dst=False)
  loc_dt += datetime.timedelta(hours=int("1600"[:2]), minutes=int("1600"[2:]))
  utc_dt = loc_dt.astimezone(pytz.utc)      
  return utc_dt.strftime('%Y-%m-%d %H:%M:%S')     

In [35]:
from google.colab import drive

drive.mount('/content/gdrive')
%cd /content/gdrive/MyDrive/chapter2/


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/MyDrive/chapter2


In [36]:
!pip install findspark 
!pip install  pyspark




Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [37]:
import findspark
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[1]").appName("architectinaction.com").getOrCreate()

In [38]:
import pandas as pd
airports = pd.read_csv('airports.csv',low_memory=False,nrows=3)
flight = pd.read_json('flights_sample.json',lines=True)


In [ ]:
!pip install TimezoneFinder

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 MB 15.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached setuptools-67.4.0-py3-none-any.whl (1.1 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 46.6 MB/s eta 0:00:00
  Created wheel for TimezoneFinder: filename=timezonefinder-6.1.9-cp38-cp38-manylinux_2_31_x86_64.whl size=46080695 sha256=d4fb469b3b6aa0500293393fc6054eced9211d35708ee1af4e9334d0d8c21c8a
  Stored in directory: /root/.cache/pip/wheels/88/c4/5c/06aa71590b3f2afef46c2bcedeac1999692037659d66e5ce9d
Successfully built TimezoneFinder
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successfully uninstalled setuptools-57.4.0
ERROR: pip's dependency resolver does not curre

In [39]:
from timezonefinder import TimezoneFinder
import pandas as pd

my_func = TimezoneFinder().timezone_at  #Note the no parenthesis on the function call!

airports['TIMEZONE'] = airports.apply(lambda x: my_func(lng=x['LONGITUDE'], lat=x['LATITUDE']),axis=1)
airports

,AIRPORT_SEQ_ID,AIRPORT_ID,AIRPORT,DISPLAY_AIRPORT_NAME,DISPLAY_AIRPORT_CITY_NAME_FULL,AIRPORT_WAC_SEQ_ID2,AIRPORT_WAC,AIRPORT_COUNTRY_NAME,AIRPORT_COUNTRY_CODE_ISO,AIRPORT_STATE_NAME,...,LON_MINUTES,LON_SECONDS,LONGITUDE,UTC_LOCAL_TIME_VARIATION,AIRPORT_START_DATE,AIRPORT_THRU_DATE,AIRPORT_IS_CLOSED,AIRPORT_IS_LATEST,Unnamed: 32,TIMEZONE
0,1000101,10001,01A,Afognak Lake Airport,"Afognak Lake, AK",101,1,United States,US,Alaska,...,54,24,-152.906667,NaN,2007-07-01,NaN,0,1,NaN,America/Anchorage
1,1000301,10003,03A,Bear Creek Mining Strip,"Granite Mountain, AK",101,1,United States,US,Alaska,...,4,18,-161.071667,NaN,2007-07-01,NaN,0,1,NaN,America/Anchorage
2,1000401,10004,04A,Lik Mining Camp,"Lik, AK",101,1,United States,US,Alaska,...,10,0,-163.166667,NaN,2007-07-01,NaN,0,1,NaN,America/Nome


In [40]:
sparkAirportsDF=spark.createDataFrame(airports)
sparkFlightDF=spark.createDataFrame(flight)

In [ ]:
sparkAirportsDF.show(10)

+--------------+----------+-------+--------------------+------------------------------+-------------------+-----------+--------------------+------------------------+------------------+------------------+------------------+------------------+--------------+-----------------------------+-----------------------+---------------+-----------+--------------+-----------+-----------+-----------+-----------+--------------+-----------+-----------+-------------+------------------------+------------------+-----------------+-----------------+-----------------+-----------+-----------------+
|AIRPORT_SEQ_ID|AIRPORT_ID|AIRPORT|DISPLAY_AIRPORT_NAME|DISPLAY_AIRPORT_CITY_NAME_FULL|AIRPORT_WAC_SEQ_ID2|AIRPORT_WAC|AIRPORT_COUNTRY_NAME|AIRPORT_COUNTRY_CODE_ISO|AIRPORT_STATE_NAME|AIRPORT_STATE_CODE|AIRPORT_STATE_FIPS|CITY_MARKET_SEQ_ID|CITY_MARKET_ID|DISPLAY_CITY_MARKET_NAME_FULL|CITY_MARKET_WAC_SEQ_ID2|CITY_MARKET_WAC|LAT_DEGREES|LAT_HEMISPHERE|LAT_MINUTES|LAT_SECONDS|   LATITUDE|LON_DEGREES|LON_HEMISPHERE|L

In [ ]:
sparkFlightDF.show(10)

+--------------+----------+-------+--------------------+------------------------------+-------------------+-----------+--------------------+------------------------+------------------+------------------+------------------+------------------+--------------+-----------------------------+-----------------------+---------------+-----------+--------------+-----------+-----------+-----------+-----------+--------------+-----------+-----------+-------------+------------------------+------------------+-----------------+-----------------+-----------------+-----------+
|AIRPORT_SEQ_ID|AIRPORT_ID|AIRPORT|DISPLAY_AIRPORT_NAME|DISPLAY_AIRPORT_CITY_NAME_FULL|AIRPORT_WAC_SEQ_ID2|AIRPORT_WAC|AIRPORT_COUNTRY_NAME|AIRPORT_COUNTRY_CODE_ISO|AIRPORT_STATE_NAME|AIRPORT_STATE_CODE|AIRPORT_STATE_FIPS|CITY_MARKET_SEQ_ID|CITY_MARKET_ID|DISPLAY_CITY_MARKET_NAME_FULL|CITY_MARKET_WAC_SEQ_ID2|CITY_MARKET_WAC|LAT_DEGREES|LAT_HEMISPHERE|LAT_MINUTES|LAT_SECONDS|   LATITUDE|LON_DEGREES|LON_HEMISPHERE|LON_MINUTES|LON_SEC

In [ ]:
sparkAirportsPandas = sparkAirportsDF.toPandas()
sparkAirportsPandas

,AIRPORT_SEQ_ID,AIRPORT_ID,AIRPORT,DISPLAY_AIRPORT_NAME,DISPLAY_AIRPORT_CITY_NAME_FULL,AIRPORT_WAC_SEQ_ID2,AIRPORT_WAC,AIRPORT_COUNTRY_NAME,AIRPORT_COUNTRY_CODE_ISO,AIRPORT_STATE_NAME,...,LON_HEMISPHERE,LON_MINUTES,LON_SECONDS,LONGITUDE,UTC_LOCAL_TIME_VARIATION,AIRPORT_START_DATE,AIRPORT_THRU_DATE,AIRPORT_IS_CLOSED,AIRPORT_IS_LATEST,Unnamed: 32
0,1000101,10001,01A,Afognak Lake Airport,"Afognak Lake, AK",101,1,United States,US,Alaska,...,W,54,24,-152.906667,NaN,2007-07-01,NaN,0,1,NaN
1,1000301,10003,03A,Bear Creek Mining Strip,"Granite Mountain, AK",101,1,United States,US,Alaska,...,W,4,18,-161.071667,NaN,2007-07-01,NaN,0,1,NaN
2,1000401,10004,04A,Lik Mining Camp,"Lik, AK",101,1,United States,US,Alaska,...,W,10,0,-163.166667,NaN,2007-07-01,NaN,0,1,NaN


In [41]:
sparkAirportsDF=spark.createDataFrame(airports)
sparkFlightDF=spark.createDataFrame(flight)

In [42]:
import pyspark
from pyspark.sql.functions import *
joindf=sparkAirportsDF.alias("sparkAirportsDF").join(sparkFlightDF.alias("sparkFlightDF"), \
    col("sparkAirportsDF.AIRPORT_SEQ_ID") == col("sparkFlightDF.ORIGIN_AIRPORT_SEQ_ID"),"inner") 
joindf .show(truncate=False)

+--------------+----------+-------+--------------------+------------------------------+-------------------+-----------+--------------------+------------------------+------------------+------------------+------------------+------------------+--------------+-----------------------------+-----------------------+---------------+-----------+--------------+-----------+-----------+-----------+-----------+--------------+-----------+-----------+-------------+------------------------+------------------+-----------------+-----------------+-----------------+-----------+-----------------+----------+--------------+---------------------+------+-------------------+----+------------+--------+---------+--------+----------+---------+-------+------------+--------+---------+---------+--------+--------+
|AIRPORT_SEQ_ID|AIRPORT_ID|AIRPORT|DISPLAY_AIRPORT_NAME|DISPLAY_AIRPORT_CITY_NAME_FULL|AIRPORT_WAC_SEQ_ID2|AIRPORT_WAC|AIRPORT_COUNTRY_NAME|AIRPORT_COUNTRY_CODE_ISO|AIRPORT_STATE_NAME|AIRPORT_STATE_CODE|AIRP

In [ ]:

def upperCase(str):
    return str.upper()

In [ ]:
upperCaseUDF = udf(lambda z:upperCase(z),StringType())   
columns = ["Seqno","Name"]
data = [("1", "john jones"),
    ("2", "tracey smith"),
    ("3", "amy sanders")]

df = spark.createDataFrame(data=data,schema=columns)

df.show(truncate=False)
df.withColumn("Cureated Name", upperCaseUDF(col("Name"))) \
  .show(truncate=False)

+-----+------------+
|Seqno|Name        |
+-----+------------+
|1    |john jones  |
|2    |tracey smith|
|3    |amy sanders |
+-----+------------+

+-----+------------+-------------+
|Seqno|Name        |Cureated Name|
+-----+------------+-------------+
|1    |john jones  |JOHN JONES   |
|2    |tracey smith|TRACEY SMITH |
|3    |amy sanders |AMY SANDERS  |
+-----+------------+-------------+



In [43]:
import datetime, pytz

@udf(returnType=StringType())
def convert_to_utc(date,time_col, time_zone):

  loc_tz = pytz.timezone(time_zone)
  loc_dt = loc_tz.localize(datetime.datetime.strptime(date, '%Y-%m-%d'), is_dst=False)
  loc_dt += datetime.timedelta(hours=int(time_col[:2]), minutes=int(time_col[2:]))
  utc_dt = loc_dt.astimezone(pytz.utc)      
  return utc_dt.strftime('%Y-%m-%d %H:%M:%S')     
      
    

In [45]:

joindf = joindf.withColumn("CRS_DEP_TIME",joindf["CRS_DEP_TIME"].cast(StringType()))

joindf = joindf.withColumn("DEP_TIME",joindf["DEP_TIME"].cast(StringType()))
joindf.select("DEP_TIME").dtypes

[('DEP_TIME', 'string')]

In [47]:

joindf=joindf.withColumn("CRS_DEP_TIME_", convert_to_utc("FL_DATE","CRS_DEP_TIME","TIMEZONE"))
joindf=joindf.withColumn("DEP_TIME", convert_to_utc("FL_DATE","DEP_TIME","TIMEZONE"))
joindf.show(truncate=False)

+--------------+----------+-------+--------------------+------------------------------+-------------------+-----------+--------------------+------------------------+------------------+------------------+------------------+------------------+--------------+-----------------------------+-----------------------+---------------+-----------+--------------+-----------+-----------+-----------+-----------+--------------+-----------+-----------+-------------+------------------------+------------------+-----------------+-----------------+-----------------+-----------+-----------------+----------+--------------+---------------------+------+-------------------+----+------------+-------------------+---------+--------+----------+---------+-------+------------+--------+---------+---------+--------+--------+-------------------+-------------------+
|AIRPORT_SEQ_ID|AIRPORT_ID|AIRPORT|DISPLAY_AIRPORT_NAME|DISPLAY_AIRPORT_CITY_NAME_FULL|AIRPORT_WAC_SEQ_ID2|AIRPORT_WAC|AIRPORT_COUNTRY_NAME|AIRPORT_COUNTRY_

In [ ]:
#load csv and json
#load dataframe in to db
#save dataframe as file


In [34]:

joindf1=joindf.withColumn("EVENT_TYPE", when(col("DEP_TIME") >0,"departed" ))
joindf2=joindf.withColumn("EVENT_TYPE",when(col("ARR_TIME")>0,"arrived" ) )
joindf=joindf1.union(joindf2)
joindf.show(truncate=False)

PythonException: ignored